#

In [31]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()
api_key = os.getenv("WEATHER_API_KEY")

In [34]:

def fetch_city_coordinates(api_key, city):
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=5&appid={api_key}"
    response = requests.get(url)
    return response.json()

city_coordinates = fetch_city_coordinates(api_key, city="New York")
print(json.dumps(city_coordinates, indent=4))

[
    {
        "name": "New York County",
        "local_names": {
            "pt": "Nova Iorque",
            "zh": "\u7ebd\u7ea6/\u7d10\u7d04",
            "fa": "\u0646\u06cc\u0648\u06cc\u0648\u0631\u06a9",
            "ja": "\u30cb\u30e5\u30fc\u30e8\u30fc\u30af",
            "uk": "\u041d\u044c\u044e-\u0419\u043e\u0440\u043a",
            "kn": "\u0ca8\u0ccd\u0caf\u0cc2\u0caf\u0cca\u0cb0\u0ccd\u0c95\u0ccd",
            "gl": "Nova York",
            "ko": "\ub274\uc695",
            "de": "New York",
            "oc": "N\u00f2va York",
            "ar": "\u0646\u064a\u0648\u064a\u0648\u0631\u0643",
            "ru": "\u041d\u044c\u044e-\u0419\u043e\u0440\u043a",
            "es": "Nueva York",
            "fr": "New York",
            "cs": "New York",
            "en": "New York",
            "el": "\u039d\u03ad\u03b1 \u03a5\u03cc\u03c1\u03ba\u03b7",
            "ca": "Nova York",
            "he": "\u05e0\u05d9\u05d5 \u05d9\u05d5\u05e8\u05e7",
            "pl": "Nowy Jork",
   

In [41]:
def fetch_air_pollution_data(api_key, lat, lon, start, end):
    url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}"
    response = requests.get(url)
    return response.json()

air_pollution_data = fetch_air_pollution_data(api_key, lat="40.7127281", lon="-74.0060152", start=1606488670, end=1606747870)
#print(json.dumps(air_pollution_data, indent=4))

In [43]:
# Organizing data into a DataFrame

data_list = air_pollution_data['list']
df = pd.DataFrame(data_list)

# Converting UNIX timestamps to datetime
df['dt'] = pd.to_datetime(df['dt'], unit='s')

# Expanding the 'main' and 'components' dictionaries into separate columns
df_main = df['main'].apply(pd.Series)
df_components = df['components'].apply(pd.Series)

# Concatenating all data into one DataFrame
df = pd.concat([df.drop(['main', 'components'], axis=1), df_main, df_components], axis=1)
df.head()

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2020-11-27 15:00:00,4,1134.87,135.90,29.13,2.66,6.32,33.99,50.53,4.94
1,2020-11-27 16:00:00,2,620.84,51.41,31.53,9.21,14.42,16.51,24.08,3.17
2,2020-11-27 17:00:00,2,567.44,39.79,39.07,12.88,17.17,15.99,22.75,3.36
3,2020-11-27 18:00:00,2,547.41,33.98,46.61,13.05,18.84,15.55,22.25,3.61
4,2020-11-27 19:00:00,2,527.38,28.16,48.67,11.80,15.26,13.43,19.61,3.58


# Analysis

Let's analyze the data to find patterns. Specifically, let's investigate:

- Average levels of each pollutant over the time period
- Days with the highest pollution levels
- Days with the lowest pollution levels
- How pollution levels vary throughout the day.